# Authentic Company Data: Requirements for organization SSI wallets

In this presentation, we will:

* Describe the overall stack and provide a high level overview of the solution
* Dig deeper into each solution component and explain why it is important
* Initiate all the required components for Bolagsverket to interact with Aktiebolaget AB
* Show how Bolagsverket can issue a Verifiable Credential containing company registration data to Aktiebolaget AB
    1. Introduce the VC formats
    2. Demonstrate the ZKP ones

## 1. Input to the overall architecture - ToIP

The Trust over IP stack has two tracks: Technology and Governance. The technology stack contains all the technical components that make it possible to exchange verifiable data between two actors. The governance track contains the rules and policies that govern each layer of the technical solution.

<img src = 'fig/toip.png' width = 700>

Below, we look specifically at the technology stack (layers 1-3) and run everything using the default governance settings and in "God mode".

### 1.1. Layer 1: DID Registry and DID Method

*DIDs are a special kind of identifiers that facilitate interactor exchanges. They can be published in a DID registry. A specific DID method explains how to create, update, and delete a DID and its associated DID Document.*

#### DIDs
To understand a DID registry and a DID Method, we must understand what a **D**ecentralized **ID**entifier is. A DID is a type of identifier that resolves into a DID Document, which contains all the information required to create a connection to the DID controller.

<img src = 'fig/did.svg' width = 400>

The standard elements of a DID Document include (or can include):

1. The DID itself as the document identifier
2. A set of public keys 
3. A set of authentication methods
4. A set of service endpoints for interaction
5. A timestamp to enable audits
6. A signature to ensure integrity

#### DID Registration

To enable trust in the identity ecosystem, certain actors register their DID on a public data registry. Note that the properties of the data registry determines the technical trust in the DID registration. For instance, actors may wish to use publicly verifiable hash pointers for retrieval.

<img src = 'fig/diddoc.svg' width = 600>

#### The DID Method.

The DID method is a specification for how to create, update, delete a DID Document. Each VDR has a specific DID method, some have several. To learn about the DID method for a specific VDR, see the [W3C did spec registry](https://w3c.github.io/did-spec-registries/#did-methods).

### 1.2. Layer 1: Design input

*Use an Indy, Aries, Ursa projects based approach. Well documented and best development tool support.*

**Selection criteria:**

1. a VDR specifically designed for digital identity data so that we do not have to create things at the DID registry level
2. a commonly used DID method that works with our VDR
3. a solution with a proven deployment
4. a well documented solution with a lot of open information and guides

For our purposes, the British Colombia (BC) project "The Verifiable Organizations Network" ([VON](https://vonx.io/)) checks all boxes. It builds on Hyperledger Indy, a purpose built VDR for identity. It uses `sov` as a DID Method, which works well with the Indy VDR. The potential has been successfully demonstrated resulting in a [deployed network](https://sovrin-mainnet-browser.vonx.io/) and a [data registry containing organization identifiers](https://orgbook.gov.bc.ca/search). Finally, the BC team has documented their results and prepared docker containers for easy concept testing:

* The [von network github](https://github.com/bcgov/von-network)
* The [OrgBook credential registry github](https://github.com/bcgov/TheOrgBook)
* The [GreenLight decentralized workflow github](https://github.com/bcgov/greenlight)

**Alternative candidates:**

1. Hyperledger Besu and the EBSI network. Decided against as several key features (e.g., ZKP based revocation) were lacking. Not as well documented as the BC project.
2. Public Ethereum based. Decided against as the existing guides were not as purposeful as the BC project one was.

**Implications of design choice:**

1. The goal is to launch a solution on EBSI. Currently, EBSI does not support Indy ledgers.
    * Potential solution 1: Implement support for Indy ledger on the Swedish EBSI dev node
    * Potential solution 2: Focus on verifiable credential formats that do not require a DLT for the credential itself
2. The BC gov project support multiple different credential formats and allows us to experiment and learn about verifiable credentials without being bound to a specific VDR. The VDR is kind of a plugin anyway so it does not matter at this early stage.

### 1.3. Layer 2: DIDComm and agent to agent data exchange

*Layer 2 contains all the protocols and enablers for agent to agent data exchanges. We rely on the Aries protocol suite.*

Today, there is really only one viable basis for an easy POC environment: [Hyperledger Aries](https://www.hyperledger.org/use/aries#:~:text=Hyperledger%20Aries%20provides%20a%20shared,peer%2Dto%2Dpeer%20interactions.). Aries provides a toolkit designed for agent interactions centered around verifiable digital credentials. Aries provides a full suite of protocols required for every possible interaction and contains a full set of protocols to support multiple different Verifiable Credential formats (including those with advanced privacy and user control features as compared [here](https://github.com/PeterAltmann/SSIdemo/blob/main/VC_formats/Verifiable-Credentials-Flavors-Explained-Infographic.pdf)).

Aries also loads Hyperledger Ursa, a complete cryptography library for functions and algorithms.

### 1.4. Layer 2: Design input

There is really no viable alternative to Hyperledger Aries and Hyperledger Ursa.

### 1.5. Layer 3: Credential exchanges

*There exist four different formats for verifiable credentials. We will work with the more privacy oriented ones. This [infographic](Verifiable-Credentials-Flavors-Explained-Infographic.pdf) is the arguably the best VC format comparison that exists.*

While many parts of the credential exchange is specified in Aries, we must agree on the specific credential format to use. There are [three options listed](https://www.w3.org/TR/vc-imp-guide/#proof-formats) by W3C, and there is the additional option of AnonCreds. Each option has benefits and drawbacks.

**JSON-JWT**

* Simplest solution
    * Attributes are stored as key value pairs
* Lacks privacy features
* Lacks semantic interoperability

**JSON-LD with LD signature**

* Relatively simple
    * Attributes are stored as key value pairs
    * Schema is stored anywhere
* Lacks privacy features
* Provides semantic interoperability using a `@context` and pointers for each key. Time consuming to construct.

**ZKP-CL**

* Very complicated with several components
    * Attributes are stored as key value pairs
    * Definitions are stored on VDR
    * Schema is stored on VDR
* Privacy features
    * Selective disclosure
    * Derivation
    * Anti correlation
* Lacks semantic interoperability

**JSON-LD ZKP with BBS+**

* Complicated
    * Attributes are stored as key value pairs
    * Schema is stored anywhere
* Privacy features
    * Selective disclosure
    * Derivation theoretically possible but no wide spread implementation yet
    * Anti correlation
* Provides semantic interoperability using a `@context` and pointers for each key. Time consuming to construct.

Note that the W3C specifications do not include **ZKP-CL**. However, **ZKP-CL** is the most commonly adopted VC format due to its early support (it was the first credential format).

### 1.6. Layer 3: Design input

*In this demo, we will mainly use the AnonCreds format, which is based on ZKP-CL (a multi messaging signature scheme with ZKP support). We also demonstrate JSON-LD ZKP with BBS+*

**Selection:** 

* We start with ZKP-CL simply because it is what the AnonCreds format uses. The AnonCreds format is the oldest and most well documented of the VC formats that supporWhilet enhanced privacy features. While complicated, most of the complicated features are handled by Indy and Aries.
* We will also look at the JSON-LD ZKP with BBS+ format. It is supported by the W3C standard and it has several important privacy features. It is also likely to be the format that the DIF / W3C community will lend most support to. It is a bit harder to implement than AnonCreds because we have to rely on linked data principles, which take a lot of time.

**Alternative candidates**

None exist. Yet. Solid ones are on the way.

**Implications of design choices**

1. AnonCreds will help us understand Layer 1 well. It will also help us understand the possible uses of a VDR in credential exchanges. Since Indy, Aries, and Ursa handle a lot of the complicated aspects of ZKP-CL, we can start experimenting and demoing early since we only need to define simple key value pairs for our attributes. AnonCreds can also help us understand a lot about the more advanced privacy features.
2. However, AnonCreds are not listed on W3C as a supported credential format. JSON-LD ZKP with BBS+ is.
3. JSON-LD with BBS+ has several yet unresolved issues and several key features are not yet implemented (but are theoretically possible), e.g., revocation.

## 2. Important to know before the AnonCreds demo

### 2.1. The ZKP-CL credential format

Key features: 

* **Multi message** signing
    * I explain the math behind the Strong RSA based CL scheme [here](https://github.com/PeterAltmann/SSIdemo/blob/main/VC_formats/math_CL.ipynb)
* **EC pairings** based
    * Decisional Diffie-Hellman is easy.
        * In a cyclic group of order $q$, with generator $g$, given $(g^{a}, g^{b})$ distinguish between  $g^{ab}$ and $g^{r}$ with $(a,b) \in \mathbb{Z}_q$
    * Computational Diffie-Hellman remains hard
        * Given $(g^{a}, g^{b})$, compute $g^{ab}$
    * The actual math of how the pairing is done and why it works is presently way beyond me
* Uses VDR to post **schema**
* Uses VDR to post **credential definition**, ie., a list of public keys used to sign each key value pair in the corresponding schema
* Allows for **selective disclosure** (I did not test yet but know how the math works)
* Allows for **derivation** (I did not test yet but I know the math behind some ZKP work that supports derivation)

We begin with a high level overview of the credential format

<img src='fig/zkp-cl.png'>



### 2.2. Choice of Aries framework

*We will use `aca-py` as our agent framework. It has support for all the VC formats we want, is well documented, is server oriented, and is written in Python (easier for me to understand what it does).*

The protocol specifications in Aries are quite extensive ([index on github](https://github.com/hyperledger/aries-rfcs/blob/main/index.md)). To work with the protocols, we will use an Aries framework, which implements several of the protocols and makes it easier to develop a wallet.

There exist multiple different frameworks (a complete list can be found [here](http://aries-interop.info/)). Our choice is informed by:

* We would prefer a solution that supports both AIP 1 and AIP 2 (Aries Interoperability Profiles). 
    * The AnonCreds format is in AIP 1. 
    * The W3C format is in AIP 2. 
* We need something focused on enterprise wallets
* We want something that is well documented
* I am most comfortable with Python

The ACA-Py (Aries Cloud Agent Python) framework supports both AIP 1 and AIP 2 and is specifically designed for cloud environments. It has a very extensive documentation since it is featured both in the BC project (they contributed the initial implementation) and it is part of the Hyperledger course [LFS173x](https://training.linuxfoundation.org/training/becoming-a-hyperledger-aries-developer-lfs173/).

With ACA-Py we will have an agent framework that handles the various Aries protocols (content focused) we need, as well as the various DIDCom protocols (delivery focused) we need.

### 2.3. The controller - Framework interactions

<img src = 'fig/agent-controller.png'>

The ACA-Py agent instance implements parts of the Aries framework, specifically it:

1. Determines the connection type
2. Creates protocol state objects
3. Stores protocol state objects
4. Sends webhooks about states and state changes

The controller receives events and posts requests to the ACA-Py agent instance using REST API. It:

1. Encodes the business logic that defines how different state objects should be handled (eg., what to do when a credential offer is received).
2. Retrieves information from the protocol
3. Constructs and sends HTTP requests to the ACA-Py administrative endpoint.

### 2.4. A platform built on Aries, Ursa, and Indy

Below is a high level schematic of the various solution components and how Aries, Ursa and Indy together enable verifiable data exchanges.

<img src = 'fig/hl-platform.png'>

### 2.5. Standards

* **DID** are being standardized under the Decentralized Identity Foundation.
* **Decentralized Key Management Systems** are being standardized under OASIS
* The **DID Auth** method for proving control of a DID is being standardized under DIF and the IETF
* **Verifiable Credentials** is a standard under W3C
* The cryptography is being standardized (unclear when in Europe)
* The Aries RFC

# Demonstration of a verifiable Authentic Company Data exchange platform

## 1. Demo A - The AnonCreds VC flow

<img src='fig/toip-standards.png'>

## 2. Overview of steps for CL-ZKP VC format

Before we can get our agents to exchange data, we need to do the following steps:

1. Start a VDR and register
    * DIDs
2. Setup a tails server for the issuer so that revocation is supported (see https://github.com/PeterAltmann/SSIdemo/blob/main/math_accumulators.ipynb for an illustration of the principles behind cryptographic accumulators)
3. Start a webhook server to receive events from our agent instances
4. Configure our agents

Once the agents are configured we can continue and

5. Let Bolagsverket issue a Credential schema
6. Let Bolagsverket issue a corresponding Credential definition
7. Connect Bolagsverket with Aktiebolaget
8. Have Bolagsverket issue a VC with company registration data to Aktiebolaget
9. Check the revocation status of the VC

In [1]:
# Import required prerequisites
import os
import requests
import subprocess
import json
import time

In [2]:
# The server startup commands
cmd_indy = 'echo 123456 | sudo -S ~/von-network/manage start --wait' # The DID registry
cmd_webhook = 'cd ~/webhook.site && echo 123456 | sudo -S docker-compose up &' # The webhook server for our events
cmd_tails = 'echo 123456 | sudo -S ~/indy-tails-server/docker/manage start' # The revocation server

In [3]:
# CODE TO CLEAR OUR OLD WALLETS
fresh_start = !ls -A ~/.indy_client/wallet/
if len(fresh_start) != 0:
    !rm -rf ~/.indy_client/wallet/*

In [4]:
# CODE TO CLEAR THE SERVERS
cmd_server_down = 'echo 123456 | sudo -S -v && sudo ~/von-network/manage down && sudo ~/indy-tails-server/docker/manage down && cd ~/webhook.site && sudo docker-compose down'
cmd_indy_vdr_down = 'echo 123456 | sudo -S -v && sudo ~/von-network/manage down'
cmd_server_stop = 'echo 123456 | sudo -S -v && sudo ~/indy-tails-server/docker/manage stop && cd ~/webhook.site && sudo docker-compose stop'
# !{cmd_indy_vdr_down}

### 2.1. Start VDR and Register DID

In [5]:
os.system(cmd_indy) # Takes ~15 seconds for the nodes to sync

time.sleep(1)

VDR = "http://localhost:9000/register"
did_reg = '{"seed": "Bolagsverket00000000000000000001", "role": "TRUST_ANCHOR", "alias": "Bolagsverket"}'
r_did_reg = requests.post(url=VDR, data=did_reg)
print(r_did_reg.text)
issuer_did = r_did_reg.json()['did'] # We store the issuer DID for future use

[sudo] password for parallels: 


Using: docker-compose --log-level ERROR



Creating von_node4_1 ... 
Creating von_node2_1 ... 
Creating von_webserver_1 ... 
Creating von_node1_1     ... 
Creating von_node3_1     ... 
Creating von_webserver_1 ... done
Creating von_node1_1     ... done
Creating von_node4_1     ... done
Creating von_node2_1     ... done
Creating von_node3_1     ... done


waiting for ledger to start.............Want to see the scrolling container logs? Run "./manage logs"
{
  "did": "h2aCQwapPeZtdoHH1VAh6",
  "seed": "Bolagsverket00000000000000000001",
  "verkey": "NpLUnDKyMhSXPfYyUEPMVwCyPys81LaoEMQjXbsfXkn"
}


In [6]:
issuer_did = 'h2aCQwapPeZtdoHH1VAh6'

**Layer 1 events**:

1. Started an Indy network to act as a VDR
2. Created a DID for Bolagsverket using a seed
3. Registered that DID in the VDR

http://localhost:9000/browse/domain?page=1&query=h2aCQwapPeZtdoHH1VAh6

### 2.2. Start tails server for revocation purposes

In [7]:
# The tails server will run on http://localhost:6543
os.system(cmd_tails)

[sudo] password for parallels: Creating network "docker_tails-server" with the default driver
Creating network "docker_default" with the default driver
Creating docker_ngrok-tails-server_1 ... 
Creating docker_tails-server_1       ... 


Run './manage logs' for logs


Creating docker_tails-server_1       ... done
Creating docker_ngrok-tails-server_1 ... done


0

**Supporting events**

1. Started a server for the tails file that Bolagsverket can use for revocation.

### 2.3. Start the webhook server

A dispatched webhook appends the path component to the URL. I.e., `https://webhook.host.example` becomes `https://webhook.host.example/topic/connections` when a connection record is updated. The list of available paths are: {`/connections`, `/basicmessages`, `/forward`, `/issue_credential`, `/present_proof`}. Each webhook payload has a set of properties (see list [here](https://github.com/hyperledger/aries-cloudagent-python/blob/main/AdminAPI.md)).

In [8]:
# Start the webhook server on http://127.0.0.1:8084
os.system(cmd_webhook)

0

[sudo] password for parallels: Creating network "webhooksite_default" with the default driver
Creating webhooksite_laravel-echo-server_1 ... 
Creating webhooksite_redis_1               ... 
Creating webhooksite_webhook_1             ... 
Creating webhooksite_laravel-echo-server_1 ... done
Creating webhooksite_webhook_1             ... done
Creating webhooksite_redis_1               ... done


Attaching to webhooksite_laravel-echo-server_1, webhooksite_webhook_1, webhooksite_redis_1
redis_1                | 1:C 19 May 2022 07:29:53.004 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
redis_1                | 1:C 19 May 2022 07:29:53.004 # Redis version=6.2.6, bits=64, commit=00000000, modified=0, pid=1, just started
redis_1                | 1:C 19 May 2022 07:29:53.004 # Warning: no config file specified, using the default config. In order to specify a config file use redis-server /path/to/redis.conf
redis_1                | 1:M 19 May 2022 07:29:53.005 * monotonic clock: POSIX clock_gettime
redis_1                | 1:M 19 May 2022 07:29:53.007 * Running mode=standalone, port=6379.
redis_1                | 1:M 19 May 2022 07:29:53.007 # Server initialized
redis_1                | 1:M 19 May 2022 07:29:53.007 # WARNING overcommit_memory is set to 0! Background save may fail under low memory condition. To fix this issue add 'vm.overcommit_memory = 1' to /etc/sysctl.conf and the

**Supporting event**:

1. Started a server for receiving webhooks.
2. Visit the webhook site on http://127.0.0.1:8084 and get a unique url for each agent.

In [9]:
web_hook = 'http://127.0.0.1:8084/fa1a489e-1d94-4eb2-b405-5aae09ca9938'

### 2.4. Agent configuration

Each agent instance takes as input a series of options and argument

In [ ]:
# Bolagsverket's configuration file is
!cat ~/Bolagsverket/config_bv.yaml

In [ ]:
# Aktiebolagets's configuration file is
!cat ~/Bolagsverket/config_ab.yaml

In [10]:
# We start the Bolagsverket agent first
agent_bv = subprocess.Popen(['aca-py', 'start', '--arg-file', '~/Bolagsverket/config_bv.yaml', '--webhook-url', web_hook])
print(f'ACA-Py instance for Bolagsverket running on PID {agent_bv.pid}')

# We start the Aktiebolaget agent first
agent_ab = subprocess.Popen(['aca-py', 'start', '--arg-file', '~/Bolagsverket/config_ab.yaml'])
print(f'ACA-Py instance for Aktiebolaget running on PID {agent_ab.pid}')

ACA-Py instance for Bolagsverket running on PID 9000
ACA-Py instance for Aktiebolaget running on PID 9003


**Layer 2 support events:**

1. We have started two agent instances
    * Bolagsverket agent instance
    * Aktiebolag agent instance
2. We have setup an administrative interface for both agents
    * http://localhost:11000/api/doc for Bolagsverket
    * http://localhost:11001/api/doc for Aktiebolaget
3. The agents are running in auto mode to the greatest extent possible
4. The Bolagsverket agent has registered additional information on the VDR ([link](http://localhost:9000/browse/domain?page=1&query=h2aCQwapPeZtdoHH1VAh6))

In [ ]:
# ## CODE FOR KILLING AGENTS
# # If you have to kill agent, it is not enough to kill the aca-py process with agent_*.kill(), you have to kill the admin processes too

# pid_bv_acapy = !echo 123456 | sudo -S ss -lptn 'sport = :8000'
# pid_ab_acapy = !echo 123456 | sudo -S ss -lptn 'sport = :8001'
# pid_bv_acapy = (pid_bv_acapy.spstr).split('pid=')[1].split(',',1)[0]
# pid_ab_acapy = (pid_ab_acapy.spstr).split('pid=')[1].split(',',1)[0]

# !kill {pid_bv_acapy}
# !kill {pid_ab_acapy}
# agent_bv.kill()
# agent_ab.kill()

Everything we require is now up and running. Now we can start preparing for the actual agent interactions. As a reminder, we will now publish a schema, definitions, connect the agents, and start credential exchanges.

### 2.5. Publish a credential schema

In [11]:
## The schema data
schem_reg = '''{
  "attributes": [
  "registration_number",
  "business_name",
  "address",
  "registered_office",
  "signatory_power",
  "board_members",
  "board_chair",
  "share_capital"
  ],
  "schema_name": "e-VC-reg",
  "schema_version": "1.0"
}'''
header_json = {"Content-Type": "application/json"}
issuer_schema = "http://localhost:11000/schemas"

## Publish schema
r_schema_reg = requests.post(url=issuer_schema, headers=header_json, data=schem_reg)
schema_id = r_schema_reg.json()['schema_id'] # We store the schema_id for future use

The schema is now available at http://localhost:9000/browse/domain?page=1&query=h2aCQwapPeZtdoHH1VAh6

### 2.6. Publish a credential definition that supports revocation

In [12]:
schema_id

'h2aCQwapPeZtdoHH1VAh6:2:e-VC-reg:1.0'

In [13]:
## The cred def data. Takes about 15 seconds.
cred_def_reg = '''{
  "schema_id": "h2aCQwapPeZtdoHH1VAh6:2:e-VC-reg:1.0",
  "tag": "default",
  "revocation_registry_size": 15,
  "support_revocation": true
  }'''

header_json_cred_def = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

issuer_cred_def = "http://localhost:11000/credential-definitions"

## Publish cred_def
r_cred_def_reg = requests.post(url=issuer_cred_def, headers=header_json_cred_def, data=cred_def_reg)
r_cred_def_reg.json()

webhook_1              | 127.0.0.1 -  19/May/2022:07:31:29 +0000 "POST /index.php" 200
webhook_1              | 127.0.0.1 -  19/May/2022:07:31:29 +0000 "POST /index.php" 200
webhook_1              | [2022-05-19 07:31:29] Processing: App\Events\RequestCreated
laravel-echo-server_1  | Channel: fa1a489e-1d94-4eb2-b405-5aae09ca9938
laravel-echo-server_1  | Event: request.created
webhook_1              | [2022-05-19 07:31:29] Processed:  App\Events\RequestCreated
webhook_1              | [2022-05-19 07:31:29] Processing: App\Events\RequestCreated
webhook_1              | [2022-05-19 07:31:29] Processed:  App\Events\RequestCreated
laravel-echo-server_1  | CHANNEL fa1a489e-1d94-4eb2-b405-5aae09ca9938
laravel-echo-server_1  | Channel: fa1a489e-1d94-4eb2-b405-5aae09ca9938
laravel-echo-server_1  | Event: request.created
laravel-echo-server_1  | CHANNEL fa1a489e-1d94-4eb2-b405-5aae09ca9938
webhook_1              | 127.0.0.1 -  19/May/2022:07:31:32 +0000 "POST /index.php" 200
webhook_1            

{'credential_definition_id': 'h2aCQwapPeZtdoHH1VAh6:3:CL:8:default'}

webhook_1              | [2022-05-19 07:31:41] Processing: App\Events\RequestCreated
laravel-echo-server_1  | Channel: fa1a489e-1d94-4eb2-b405-5aae09ca9938
laravel-echo-server_1  | Event: request.created
laravel-echo-server_1  | CHANNEL fa1a489e-1d94-4eb2-b405-5aae09ca9938
webhook_1              | [2022-05-19 07:31:41] Processed:  App\Events\RequestCreated


In [14]:
cred_def_id = r_cred_def_reg.json()['credential_definition_id'] # We store the cred_def_id for future use

The `cred_def_id` is now available at http://localhost:9000/browse/domain?page=1&query=h2aCQwapPeZtdoHH1VAh6

### 2.7. A connection between agents

In [15]:
url_connection_req = 'http://localhost:11001/didexchange/create-request?their_public_did=h2aCQwapPeZtdoHH1VAh6'
data_connection_req = ''

requests.post(url=url_connection_req, headers={"Content-Type": "application/json"}, data=data_connection_req)

<Response [200]>

webhook_1              | 127.0.0.1 -  19/May/2022:07:33:09 +0000 "POST /index.php" 200
webhook_1              | 127.0.0.1 -  19/May/2022:07:33:09 +0000 "POST /index.php" 200
webhook_1              | 127.0.0.1 -  19/May/2022:07:33:09 +0000 "POST /index.php" 200
webhook_1              | 127.0.0.1 -  19/May/2022:07:33:09 +0000 "POST /index.php" 200


In [16]:
connections_active_bv = requests.get(url='http://localhost:11000/connections', headers={'accept': 'application/json'})
connections_identifiers_bv = [connections_active_bv.json()['results'][i]['connection_id'] for i in range(len(connections_active_bv.json()['results']))]

connections_active_ab = requests.get(url='http://localhost:11001/connections', headers={'accept': 'application/json'})
connections_identifiers_ab = [connections_active_ab.json()['results'][i]['connection_id'] for i in range(len(connections_active_ab.json()['results']))]

print(f'Bolagsverket has the following connection ID(s) to Aktiebolaget:\n{connections_identifiers_bv}')
print(f'\nAktiebolaget has the following connection ID(s) to Bolagsverket:\n{connections_identifiers_ab}')

webhook_1              | [2022-05-19 07:33:12] Processing: App\Events\RequestCreated
laravel-echo-server_1  | Channel: fa1a489e-1d94-4eb2-b405-5aae09ca9938
laravel-echo-server_1  | Event: request.created
laravel-echo-server_1  | CHANNEL fa1a489e-1d94-4eb2-b405-5aae09ca9938
webhook_1              | [2022-05-19 07:33:12] Processed:  App\Events\RequestCreated
webhook_1              | [2022-05-19 07:33:12] Processing: App\Events\RequestCreated
laravel-echo-server_1  | Channel: fa1a489e-1d94-4eb2-b405-5aae09ca9938
laravel-echo-server_1  | Event: request.created
laravel-echo-server_1  | CHANNEL fa1a489e-1d94-4eb2-b405-5aae09ca9938
webhook_1              | [2022-05-19 07:33:12] Processed:  App\Events\RequestCreated
webhook_1              | [2022-05-19 07:33:12] Processing: App\Events\RequestCreated
Bolagsverket has the following connection ID(s) to Aktiebolaget:
['6b0dc55a-f301-4a3e-9e96-8a37bee42a6c']

Aktiebolaget has the following connection ID(s) to Bolagsverket:
['bb69fbca-16a0-47dc-823c

In [ ]:
# connection_id_ab = !curl -X GET "http://localhost:11001/connections" -H  "accept: application/json" | jq | grep '"connection_id":'
# connection_id_ab = (connection_id_ab.spstr).split('connection_id":',1)[1].split(" ",1)[1].split('"',1)[1].split('"',-1)[0]

# connection_id_bv = !curl -X GET "http://localhost:11000/connections" -H  "accept: application/json" | jq | grep '"connection_id":'
# connection_id_bv = (connection_id_bv.spstr).split('connection_id":',1)[1].split(" ",1)[1].split('"',1)[1].split('"',-1)[0]

# assert connection_id_ab != connection_id_bv

**Layer 2 event**

1. The two agents are now connected and ready to exchange data

### 2.8. The Verifiable Credential exchange

In [18]:
#Aktiebolaget does this 
# Copy the Aktiebolaget's connection ID to connection_id

holder_vc_req = "http://localhost:11001/issue-credential-2.0/send-proposal"
vc_req_data = '''{
  "comment": "I want a company registration VC",
  "connection_id": "bb69fbca-16a0-47dc-823c-68d7395da88c",
  "credential_preview": {
    "@type": "issue-credential/2.0/credential-preview",
    "attributes": [
       {
        "mime-type": "text/plain",
        "name": "share_capital", 
        "value": "KSEK 50"
      },
       {
        "mime-type": "text/plain",
        "name": "registration_number", 
        "value": "556907-XXXX"
      },
      {
        "mime-type": "text/plain",
        "name": "board_chair", 
        "value": "820726-2398 Al Samed, Mohamed, Stjärnallén 40, 851 81 SUNDSVALL"
      },
      {
        "mime-type": "text/plain",
        "name": "board_members", 
        "value": "8207728-2397 Andersson, Filip, Atmosfärstigen 20, 851 81 SUNDSVALL"
      },
      {
        "mime-type": "text/plain",
        "name": "signatory_power", 
        "value": "The board of directors is entitled to sign on behalf of the company. Signatory power individually by the board members."
      },
      {
        "mime-type": "text/plain",
        "name": "business_name", 
        "value": "Företaget i Sundsvall Aktiebolag"
      },
      {
        "mime-type": "text/plain",
        "name": "address", 
        "value": "Norrskensvägen 12, 851 81 SUNDSVALL"
      },
      {
        "mime-type": "text/plain",
        "name": "registered_office", 
        "value": "Sundsvall"
      }
    ]
  },                              
  "filter": {                          
    "indy": {}
  }    
}'''

## Publish cred_def
r_vc = requests.post(url=holder_vc_req, headers=header_json_cred_def, json=json.loads(vc_req_data))
print(json.dumps(r_vc.json(),indent=2, sort_keys=True))

{
  "auto_issue": false,
  "auto_offer": false,
  "auto_remove": false,
  "by_format": {
    "cred_proposal": {
      "indy": {}
    }
  },
  "connection_id": "bb69fbca-16a0-47dc-823c-68d7395da88c",
  "created_at": "2022-05-19T07:34:11.695513Z",
  "cred_ex_id": "af7281d8-a2f5-4fd8-a365-46d332d22cea",
  "cred_preview": {
    "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/2.0/credential-preview",
    "attributes": [
      {
        "mime-type": "text/plain",
        "name": "share_capital",
        "value": "KSEK 50"
      },
      {
        "mime-type": "text/plain",
        "name": "registration_number",
        "value": "556907-XXXX"
      },
      {
        "mime-type": "text/plain",
        "name": "board_chair",
        "value": "820726-2398 Al Samed, Mohamed, Stj\u00e4rnall\u00e9n 40, 851 81 SUNDSVALL"
      },
      {
        "mime-type": "text/plain",
        "name": "board_members",
        "value": "8207728-2397 Andersson, Filip, Atmosf\u00e4rstigen 20, 851 81 

In [19]:
!curl -X 'GET' \
  'http://localhost:11001/credentials' \
  -H 'accept: application/json' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   869  100   869    0     0   141k      0 --:--:-- --:--:-- --:--:--  141k
{
  "results": [
    {
      "referent": "5e7dafe0-3938-4ad8-bc5d-3679872ab012",
      "attrs": {
        "address": "Norrskensvägen 12, 851 81 SUNDSVALL",
        "board_chair": "820726-2398 Al Samed, Mohamed, Stjärnallén 40, 851 81 SUNDSVALL",
        "signatory_power": "The board of directors is entitled to sign on behalf of the company. Signatory power individually by the board members.",
        "business_name": "Företaget i Sundsvall Aktiebolag",
        "board_members": "8207728-2397 Andersson, Filip, Atmosfärstigen 20, 851 81 SUNDSVALL",
        "registration_number": "556907-XXXX",
        "registered_office": "Sundsvall",
        "share_capital": "KSEK 50"
      },
      "schema_id": "h2aCQwapPeZtdoHH1VAh6:2:e-VC-reg:1.0",
      "cred_def_id":

In [20]:
holder_credentials = requests.get(url='http://localhost:11001/credentials', headers={"Content-Type": "application/json"})
holder_credential_ids = [holder_credentials.json()['results'][i]['referent'] for i in range(len(holder_credentials.json()['results']))]
holder_credential_ids

['5e7dafe0-3938-4ad8-bc5d-3679872ab012']

**Layer 3 event**

1. The Aktiebolaget agent instance proposed a verifiable credential issuance
2. The Bolagsverket agent instance received input from the controller to simply issue everything according to the input. This is of course non standard.
3. The Bolagsverket agent checked every key and compared it to a list of possible attribute attestations it can make
4. Since the Bolagsverket agent found a match, it created the VC
5. The Bolagsverket agent assigned the VC a tails index for revocation
6. The Bolagsverket agent signs the VC and sends it to Aktiebolaget
7. Aktiebolaget stores the received VC in its wallet.

### 2.9. Check the revocation status

In [21]:
for cred_id in holder_credential_ids:
    tmp = requests.get(url='http://localhost:11001/credential/revoked/' + cred_id, headers=header_json)
    print(f'Credential {cred_id} has revoked status: {tmp.json()["revoked"]}')
    tmp = None

Credential 5e7dafe0-3938-4ad8-bc5d-3679872ab012 has revoked status: False


## 3. JSON-LD ZKP with BBS+

### 3.1. A short intro to the JSON-LD ZKP format

Compared to the CL-ZKP VCs, the process for BBS+ VCs is quite different.

1. Register the issuer on some trusted issuer list (we reuse the VDR from our above example)
2. Define a vocab and map it to each key in the VC.
3. Connect the agents (we reuse the existing connection from above)
4. Issue the VC

Note that 

* There is no registration of the schema or credential definition on the VDR! 
* We do not setup revocation options. This is because BBS+ does not yet support revocation (theoretically possible but no implementation exists).
* There is no credential preview
* Anti-correlation is a lot harder due to mandatory keys

<img src = 'fig/bbs.png' width = 450>

According to [Aries RFC0047](https://github.com/hyperledger/aries-rfcs/blob/main/concepts/0047-json-ld-compatibility/README.md), the JSON-LD support in Aries is handled by communicating the `@context` out of band as detailed in the [W3C standard section 6](https://w3c.github.io/json-ld-syntax/#interpreting-json-as-json-ld). Put differently, DIDComm messages communicate the context in the protocol definition (e.g., RFC) for the associated message type; the `@type` value gives the relevant `@context`. This lets developers focus on using regular JSON.

https://github.com/hyperledger/aries-cloudagent-python/blob/main/JsonLdCredentials.md

https://github.com/hyperledger/aries-rfcs/blob/master/features/0646-bbs-credentials/README.md

### 3.2. JSON-LD Credentials in ACA-Py

* ACA-Py supports any credential format defined in an RFC. 
* The JSON-LD VC format is one such credential format. 
* When signed with BBS+, it is possible to issue ZKP VCs.

### 3.3. Preparing JSON-LD credentials

*This is where things become tedious. JSON-LD is not as straight foward as JSON. Each key requires a mapping, and for that we need vocabs. Few exist.*

* Every property key must be mapped to an IRI (either by being an IRI or through the `@context` part). If a mapping is missing, ACA-Py's Issue Credential API will throw the following error:

> `"<x> attributes dropped. Provide definitions in context to correct. [<missing-properties>]"`
* www.schema.org has a default vocab so it will not throw an error
* The first `@context` value must always be `https://www.w3.org/2018/credentials/v1`
* BBS+ VCs require `https://w3id.org/security/bbs/v1` to be part of `@context`
* Try to make use of existing resources / vocabularies.
    * https://w3c-ccg.github.io/vaccination-vocab/
    * https://w3c-ccg.github.io/citizenship-vocab/
    * https://w3c-ccg.github.io/traceability-vocab/
    * www.schema.org
    * https://tietomallit.suomi.fi/model/ccid/
    * https://docs.dataportal.se/dcat/sv/
* The Aries RFC resources: 
    * communicates the context out of band as described [here](https://www.w3.org/TR/json-ld11/#interpreting-json-as-json-ld)
    * https://github.com/hyperledger/aries-rfcs/blob/main/concepts/0047-json-ld-compatibility/README.md
    * https://github.com/hyperledger/aries-rfcs/blob/main/features/0593-json-ld-cred-attach/README.md details the format that all agent frameworks are supposed to use
    * https://github.com/hyperledger/aries-rfcs/blob/main/features/0453-issue-credential-v2/README.md how to exchange bbs vcs
    * https://github.com/hyperledger/aries-rfcs/blob/main/features/0646-bbs-credentials/README.md NO SUPPORT FOR REVOCATION.
    * [RFC 0510: Presentation-Exchange Attachment format for requesting and presenting proofs](https://github.com/hyperledger/aries-rfcs/blob/main/features/0510-dif-pres-exch-attach/README.md) defines an attachment format based on the DIF Presentation Exchange specification.
* Signature support
    * [`Ed25519Signature2018`](https://w3c-ccg.github.io/lds-ed25519-2018/). Very well supported. No zero knowledge proofs or selective disclosure.
    * [`BbsBlsSignature2020`](https://w3c-ccg.github.io/ldp-bbs2020/). Newer. Supports zero knowledge proofs and selective disclosure. [Informational at IETF](https://datatracker.ietf.org/doc/html/draft-irtf-cfrg-pairing-friendly-curves-10). I know standards work is beginning.
* The DID Method support in ACA-Py is:
    * `did:sov` that can only be used for [`Ed25519Signature2018`](https://w3c-ccg.github.io/lds-ed25519-2018/) signatures.
    * `did:key` that can be used for both [`Ed25519Signature2018`](https://w3c-ccg.github.io/lds-ed25519-2018/) and [`BbsBlsSignature2020`](https://w3c-ccg.github.io/ldp-bbs2020/)
* JSON-LD BBS+ requires presenting certain data that may correlate the holder:
> E.g. a credential with issuanceDate of 2017-12-05T14:27:42Z could create a correlating factor. However it is against the VC Data Model to not include the property. Take this into account when issuing credentials. From [RFC 0646.](https://github.com/hyperledger/aries-rfcs/blob/main/features/0646-bbs-credentials/README.md)
    * There is ongoing work to resolve this issue: https://github.com/w3c-ccg/ldp-bbs2020/issues/37 but there is currently no good way to ensure binding (for the latest attempt cf. https://github.com/mikelodder7/commit_twin)

### 3.4 Demo time!

#### Step 1: Create a `did:key` identifier

In [28]:
# Requires https://pypi.org/project/ursa-bbs-signatures/
bbs_key = requests.post(
    url='http://localhost:11000/wallet/did/create',
    headers={'accept': 'application/json', 'Content-Type': 'application/json'},
    json=json.loads('{"method": "key", "options": {"key_type": "bls12381g2"}}'))

bbs_key.json()

{'result': {'did': 'did:key:zUC7JfCirtQYsKSspyPGkzYnaGj5uEo4siap3EMtBzWiGpD8X4qyLFK2cHRGo3JkRqnPZpqkwdiGWifv9PNaBSGz9zZJ5f2LFtjkZgnpJnbiaE9N6JaPsuQUJZ7KPotDnFQyi8G',
  'verkey': '24peqoCkKWhafR2iRb3raGyNAexUrqk2tAqn4i93uXvrL7TtLdausoqTbe3xVuYEkRcRsuDrT8q7xCWeT2AwGL32aHK8EJFcGJ36RQZxr6H1VjQEnnyEN8khLvd9LChG7DWx',
  'posture': 'wallet_only',
  'key_type': 'bls12381g2',
  'method': 'key'}}

In [29]:
## We check existing keys
keys = requests.get(
    url='http://localhost:11000/wallet/did',
    headers={'accept': 'application/json'})

keys.json()['results']

[{'did': 'h2aCQwapPeZtdoHH1VAh6',
  'verkey': 'NpLUnDKyMhSXPfYyUEPMVwCyPys81LaoEMQjXbsfXkn',
  'posture': 'posted',
  'key_type': 'ed25519',
  'method': 'sov'},
 {'did': '8JV46un73FLt5VYcYZQojW',
  'verkey': '4yp9w3hNm9J5FznJP3o9tSF7HMGMEgmCFoUpyUDynZHd',
  'posture': 'wallet_only',
  'key_type': 'ed25519',
  'method': 'sov'},
 {'did': 'did:key:zUC7JfCirtQYsKSspyPGkzYnaGj5uEo4siap3EMtBzWiGpD8X4qyLFK2cHRGo3JkRqnPZpqkwdiGWifv9PNaBSGz9zZJ5f2LFtjkZgnpJnbiaE9N6JaPsuQUJZ7KPotDnFQyi8G',
  'verkey': '24peqoCkKWhafR2iRb3raGyNAexUrqk2tAqn4i93uXvrL7TtLdausoqTbe3xVuYEkRcRsuDrT8q7xCWeT2AwGL32aHK8EJFcGJ36RQZxr6H1VjQEnnyEN8khLvd9LChG7DWx',
  'posture': 'wallet_only',
  'key_type': 'bls12381g2',
  'method': 'key'},
 {'did': 'did:key:zUC7JgnCh2VkkHbJYQzbva8oSeo5GDNqJyzWWtN36tkd8MU7WRNj8hRQoDG1y31JUvvZp9ZgeY5cqJnmotKSFpSRFZQ3NBy8ogXuh77dvxGgKYs4fq37W2G8qMW7suB8c6MHrFR',
  'verkey': '24rEKdLqXPfj68UKkkdSb9MSA1w4dH1SaeVnDd3HpPU7K6pR6S32FzmJLp3f3xdNvfw9oc8DoTREos1bK6Z6hRbsKZr52r3QRRTRF2jdoqbjCJvhWR662fZ69Q

#### Issue the VC
BBS+ VCs are only possible with `/issue-credential-2.0`

We will issue a vaccination proof to Aktiebolaget. Which is a bit weird but ...

In [30]:
print(f"Connection id to use:\n{connections_identifiers_bv[0]}\n\nKey to use:\n{keys.json()['results'][2]['did']}")

Connection id to use:
6b0dc55a-f301-4a3e-9e96-8a37bee42a6c

Key to use:
did:key:zUC7JfCirtQYsKSspyPGkzYnaGj5uEo4siap3EMtBzWiGpD8X4qyLFK2cHRGo3JkRqnPZpqkwdiGWifv9PNaBSGz9zZJ5f2LFtjkZgnpJnbiaE9N6JaPsuQUJZ7KPotDnFQyi8G


In [31]:
#Follows https://ec.europa.eu/health/system/files/2021-06/covid-certificate_json_specification_en_0.pdf
# REMEMBER TO UPDATE CONNECTION ID AND THE ISSUER!
bbs_bc_vaccination_data = '''{
  "connection_id": "6b0dc55a-f301-4a3e-9e96-8a37bee42a6c",
  "filter": {
    "ld_proof": {
      "credential": {
        "@context": [
          "https://www.w3.org/2018/credentials/v1",
          "https://w3id.org/vaccination/v1",
          "https://w3id.org/security/bbs/v1"
        ],
        "type": ["VerifiableCredential", "VaccinationCertificate"],
        "id": "urn:uvci:01:NL:187/37512422923",
        "name": "COVID-19 Vaccination Certificate",
        "description": "COVID-19 Vaccination Certificate",
        "issuanceDate": "2019-12-03T12:19:52Z",
        "expirationDate": "2029-12-03T12:19:52Z",
        "issuer": "did:key:zUC7JfCirtQYsKSspyPGkzYnaGj5uEo4siap3EMtBzWiGpD8X4qyLFK2cHRGo3JkRqnPZpqkwdiGWifv9PNaBSGz9zZJ5f2LFtjkZgnpJnbiaE9N6JaPsuQUJZ7KPotDnFQyi8G",
        "credentialSubject": {
        "type": "VaccinationEvent",
        "batchNumber": "1183738569",
        "administeringCentre": "MoH",
        "healthProfessional": "MoH",
        "countryOfVaccination": "NZ",
        "recipient": {
          "type": "VaccineRecipient",
          "givenName": "JOHN",
          "familyName": "SMITH",
          "gender": "Male",
          "birthDate": "1958-07-17"
        }, 
        "vaccine": {
            "type": "Vaccine",
            "disease": "COVID-19",
            "atcCode": "J07BX03",
            "medicinalProductName": "COVID-19 Vaccine Moderna",
            "marketingAuthorizationHolder": "Moderna Biotech"
            }
        }
      },
      "options": {
        "proofType": "BbsBlsSignature2020"
      }
    }
  }
}'''

bbs_vc_vaccination_req = requests.post(url='http://localhost:11000/issue-credential-2.0/send',
             headers={'accept': 'application/json', 'Content-Type': 'application/json'},
             json=json.loads(bbs_bc_vaccination_data))

webhook_1              | 127.0.0.1 -  19/May/2022:07:37:56 +0000 "POST /index.php" 200
webhook_1              | 127.0.0.1 -  19/May/2022:07:37:56 +0000 "POST /index.php" 200
webhook_1              | 127.0.0.1 -  19/May/2022:07:37:57 +0000 "POST /index.php" 200
laravel-echo-server_1  | Channel: fa1a489e-1d94-4eb2-b405-5aae09ca9938
laravel-echo-server_1  | Event: request.created
laravel-echo-server_1  | CHANNEL fa1a489e-1d94-4eb2-b405-5aae09ca9938
laravel-echo-server_1  | Channel: fa1a489e-1d94-4eb2-b405-5aae09ca9938
laravel-echo-server_1  | Event: request.created
laravel-echo-server_1  | CHANNEL fa1a489e-1d94-4eb2-b405-5aae09ca9938
webhook_1              | [2022-05-19 07:37:57] Processing: App\Events\RequestCreated
webhook_1              | [2022-05-19 07:37:57] Processed:  App\Events\RequestCreated
webhook_1              | [2022-05-19 07:37:57] Processing: App\Events\RequestCreated
webhook_1              | [2022-05-19 07:37:57] Processed:  App\Events\RequestCreated
webhook_1            

And we will issue a diploma Aktiebolaget. Also a bit weird.

In [32]:
print(f"Connection id to use:\n{connections_identifiers_bv[0]}\n\nKey to use:\n{keys.json()['results'][2]['did']}")

Connection id to use:
6b0dc55a-f301-4a3e-9e96-8a37bee42a6c

Key to use:
did:key:zUC7JfCirtQYsKSspyPGkzYnaGj5uEo4siap3EMtBzWiGpD8X4qyLFK2cHRGo3JkRqnPZpqkwdiGWifv9PNaBSGz9zZJ5f2LFtjkZgnpJnbiaE9N6JaPsuQUJZ7KPotDnFQyi8G


In [33]:
#Below is a simple example diploma.
# REMEMBER TO UPDATE CONNECTION ID AND THE ISSUER!
bbs_vc_degree_data = '''{
  "connection_id": "6b0dc55a-f301-4a3e-9e96-8a37bee42a6c",
  "filter": {
    "ld_proof": {
      "credential": {
        "@context": [
          "https://www.w3.org/2018/credentials/v1",
          "https://www.w3.org/2018/credentials/examples/v1"
        ],
        "type": ["VerifiableCredential", "UniversityDegreeCredential"],
        "issuer": "did:key:zUC7JfCirtQYsKSspyPGkzYnaGj5uEo4siap3EMtBzWiGpD8X4qyLFK2cHRGo3JkRqnPZpqkwdiGWifv9PNaBSGz9zZJ5f2LFtjkZgnpJnbiaE9N6JaPsuQUJZ7KPotDnFQyi8G",
        "issuanceDate": "2022-02-08T12:00:00Z",
        "credentialSubject": {
          "degree": {
            "type": "BachelorDegree",
            "name": "Bachelor of Science and Arts"
          },
          "college": "Faber College"
        }
      },
      "options": {
        "proofType": "BbsBlsSignature2020"
      }
    }
  }
}'''

bbs_vc_degree_req = requests.post(url='http://localhost:11000/issue-credential-2.0/send',
             headers={'accept': 'application/json', 'Content-Type': 'application/json'},
             json=json.loads(bbs_vc_degree_data))

webhook_1              | 127.0.0.1 -  19/May/2022:07:38:31 +0000 "POST /index.php" 200
webhook_1              | 127.0.0.1 -  19/May/2022:07:38:31 +0000 "POST /index.php" 200
webhook_1              | 127.0.0.1 -  19/May/2022:07:38:31 +0000 "POST /index.php" 200
webhook_1              | [2022-05-19 07:38:33] Processing: App\Events\RequestCreated
laravel-echo-server_1  | Channel: fa1a489e-1d94-4eb2-b405-5aae09ca9938
laravel-echo-server_1  | Event: request.created
webhook_1              | [2022-05-19 07:38:33] Processed:  App\Events\RequestCreated
laravel-echo-server_1  | CHANNEL fa1a489e-1d94-4eb2-b405-5aae09ca9938
webhook_1              | [2022-05-19 07:38:33] Processing: App\Events\RequestCreated
laravel-echo-server_1  | Channel: fa1a489e-1d94-4eb2-b405-5aae09ca9938
webhook_1              | [2022-05-19 07:38:33] Processed:  App\Events\RequestCreated
laravel-echo-server_1  | Event: request.created
laravel-echo-server_1  | CHANNEL fa1a489e-1d94-4eb2-b405-5aae09ca9938
webhook_1            

The JSON-LD ZKP VCs have been issued. Let's check what Aktiebolaget has in its wallet.

In [34]:
bbs_w3c_vc = requests.post(url='http://localhost:11001/credentials/w3c',
                                   headers={'accept': 'application/json', 'Content-Type': 'application/json'},
                                   data='{}')

In [35]:
print(json.dumps(json.loads(bbs_w3c_vc.text)['results'][0]['cred_value'], indent=2, sort_keys=True))

{
  "@context": [
    "https://www.w3.org/2018/credentials/v1",
    "https://www.w3.org/2018/credentials/examples/v1",
    "https://w3id.org/security/bbs/v1"
  ],
  "credentialSubject": {
    "college": "Faber College",
    "degree": {
      "name": "Bachelor of Science and Arts",
      "type": "BachelorDegree"
    }
  },
  "issuanceDate": "2022-02-08T12:00:00Z",
  "issuer": "did:key:zUC7JfCirtQYsKSspyPGkzYnaGj5uEo4siap3EMtBzWiGpD8X4qyLFK2cHRGo3JkRqnPZpqkwdiGWifv9PNaBSGz9zZJ5f2LFtjkZgnpJnbiaE9N6JaPsuQUJZ7KPotDnFQyi8G",
  "proof": {
    "created": "2022-05-19T07:38:31.934473+00:00",
    "proofPurpose": "assertionMethod",
    "proofValue": "p+pvsHEc1He9vemNhYx4u82giQM0WhOpYMjD2xIHszScTQ1tvhp0DrhEUsTkuHrmXWW5gyksUiA5KhpqLQG9JCcuZYb4b5MqfclkgxwDm6tA1o7n2HvNmCm0HsElcNKPa2l35lxJkdRn7PjH1fnl7g==",
    "type": "BbsBlsSignature2020",
    "verificationMethod": "did:key:zUC7JfCirtQYsKSspyPGkzYnaGj5uEo4siap3EMtBzWiGpD8X4qyLFK2cHRGo3JkRqnPZpqkwdiGWifv9PNaBSGz9zZJ5f2LFtjkZgnpJnbiaE9N6JaPsuQUJZ7KPotD

In [36]:
print(json.dumps(json.loads(bbs_w3c_vc.text)['results'][1]['cred_value'], indent=2, sort_keys=True))

{
  "@context": [
    "https://www.w3.org/2018/credentials/v1",
    "https://w3id.org/vaccination/v1",
    "https://w3id.org/security/bbs/v1"
  ],
  "credentialSubject": {
    "administeringCentre": "MoH",
    "batchNumber": "1183738569",
    "countryOfVaccination": "NZ",
    "healthProfessional": "MoH",
    "recipient": {
      "birthDate": "1958-07-17",
      "familyName": "SMITH",
      "gender": "Male",
      "givenName": "JOHN",
      "type": "VaccineRecipient"
    },
    "type": "VaccinationEvent",
    "vaccine": {
      "atcCode": "J07BX03",
      "disease": "COVID-19",
      "marketingAuthorizationHolder": "Moderna Biotech",
      "medicinalProductName": "COVID-19 Vaccine Moderna",
      "type": "Vaccine"
    }
  },
  "description": "COVID-19 Vaccination Certificate",
  "expirationDate": "2029-12-03T12:19:52Z",
  "id": "urn:uvci:01:NL:187/37512422923",
  "issuanceDate": "2019-12-03T12:19:52Z",
  "issuer": "did:key:zUC7JfCirtQYsKSspyPGkzYnaGj5uEo4siap3EMtBzWiGpD8X4qyLFK2cHRGo3Jk

Play around: https://try.connect.me/